In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
from tqdm import tqdm

In [ ]:
BATCH_SIZE = 512
EPOCHS = 500

initial_learning_rate = 0.001
decay_steps = 1000  # 每隔多少个steps衰减一次
decay_rate = 0.96  # 学习率衰减因子

In [ ]:
AS_dataset = pd.read_csv('./hairpin_filter_dataset.csv', encoding='utf-8')

In [ ]:
full_X = AS_dataset.loc[::2,'freq':'S1'].to_numpy(dtype = np.float32)
full_y = AS_dataset.loc[::2,'S21r':'S21i'].to_numpy(dtype = np.float32)

In [ ]:
# 归一化
# full_X[:, 0] = (full_X[:, 0] - 0.1) / (9.0 - 0.1)
full_X[:, 1] = (full_X[:, 1] - 200) / (2000 - 200)
full_X[:, 2] = (full_X[:, 2] - 200) / (2000 - 200)
full_X[:, 3] = (full_X[:, 3] - 200) / (2000 - 200)
full_X[:, 4] = (full_X[:, 4] - 200) / (4500 - 200)
full_X[:, 5] = (full_X[:, 5] - 200) / (4500 - 200)
full_X[:, 6] = (full_X[:, 6] - 200) / (4500 - 200)
full_X[:, 7] = (full_X[:, 7] - 200) / (4500 - 200)
full_X[:, 8] = (full_X[:, 8] - 9000) / (12000 - 9000)
full_X[:, 9] = (full_X[:, 9] - 100) / (300 - 100)

In [ ]:
X_train, X_vali, y_train, y_vali = train_test_split(full_X, full_y, test_size=0.05, random_state=0)

In [ ]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset_train = dataset_train.shuffle(buffer_size=X_train.shape[0])
dataset_train = dataset_train.batch(BATCH_SIZE)
dataset_train = dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.emb = tf.keras.layers.Embedding(101, 9)
        
        self.dense0a = tf.keras.layers.Dense(units=256)
        self.dense0b = tf.keras.layers.Dense(units=256)
        
        self.dense1 = tf.keras.layers.Dense(units=640, activation=tf.nn.leaky_relu)
        
        self.dense2 = tf.keras.layers.Dense(units=736, activation=tf.nn.leaky_relu)
        self.bn2 = tf.keras.layers.BatchNormalization()
        
        self.dense3 = tf.keras.layers.Dense(units=672, activation=tf.nn.leaky_relu)
        
        self.dense4 = tf.keras.layers.Dense(units=576)
        self.bn4 = tf.keras.layers.BatchNormalization()
        
        self.dense5 = tf.keras.layers.Dense(units=32, activation=tf.nn.leaky_relu)
        self.bn5 = tf.keras.layers.BatchNormalization()
        
        self.dense6 = tf.keras.layers.Dense(units=2)

    def call(self, inputs):
        
        f = tf.cast(tf.round(inputs[:, 0] * 10), tf.int8)
        g = inputs[:, 1:]
        x1 = self.dense0a(self.emb(f))
        x2 = self.dense0b(g)
        x = x1 + x2
        
        x = self.dense1(x)

        x = self.bn2(self.dense2(x))

        x = self.dense3(x)
        
        x = self.bn4(self.dense4(x))

        x = self.bn5(self.dense5(x))

        output = self.dense6(x)
        
        return output

In [ ]:
model = MLP()

In [ ]:
learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps,
        decay_rate,
        staircase=True  # 是否以指数方式精确衰减，默认False，若为True则每隔decay_steps学习率按decay_rate衰减
    )

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
for i in range(50):
    for X, y in tqdm(dataset_train):
        with tf.GradientTape() as tape:
            y_pred = model(X)
            tr_mse = tf.reduce_mean(tf.square(y_pred - y))
        grads = tape.gradient(tr_mse, model.trainable_variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables))
        
    tr_r2 = 1 - tf.reduce_sum(tf.square(y_pred - y)) / tf.reduce_sum(tf.square(y - tf.reduce_mean(y)))
    p_y_vali = model(X_vali)
    mse_vali = tf.reduce_mean(tf.square(p_y_vali - y_vali))
    r2_vali = 1 - tf.reduce_sum(tf.square(p_y_vali - y_vali)) / tf.reduce_sum(tf.square(y_vali - tf.reduce_mean(y_vali)))
    
    print(f'epoch{i}\t training loss: {tr_mse}\t training r2: {tr_r2}\t vali loss: {mse_vali}\t vali r2: {r2_vali}')
    
    # print(f"Epoch: {i}\t geo_seen_freq_seen(train): {tr_mse}\t geo_unseen_freq_seen: {mse_geo_unseen_freq_seen}\t geo_seen_freq_unseen: {mse_geo_seen_freq_unseen}\t geo_unseen_freq_unseen: {mse_geo_unseen_freq_unseen}")
    # log.append({'epoch': i, 'geo_seen_freq_seen': tr_mse, 'geo_unseen_freq_seen': mse_geo_unseen_freq_seen, 'geo_seen_freq_unseen': mse_geo_seen_freq_unseen, 'geo_unseen_freq_unseen': mse_geo_unseen_freq_unseen})
    # print(model.emb.variables[0][45])
    # visual(model.emb.variables[0].numpy())